In [2]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [10]:
tweets = pd.read_csv(r"C:/Users/student/Documents/dataset/Tweets.csv")

In [ ]:
data = tweets[['text','airline_sentiment']]

In [13]:
data = data[data.airline_sentiment != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(data[ data['airline_sentiment'] == 'positive'].size)
print(data[ data['airline_sentiment'] == 'negative'].size)

35445
137670


In [14]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [15]:
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7732, 32) (7732, 2)
(3809, 32) (3809, 2)


In [17]:
X_val = X_train[:500]
Y_val = Y_train[:500]

In [18]:
partial_X_train = X_train[500:]
partial_Y_train = Y_train[500:]

In [19]:
batch_size = 512
history = model.fit(partial_X_train, 
                    partial_Y_train, 
                    epochs = 10, 
                    batch_size=batch_size, 
                    validation_data=(X_val, Y_val))

Train on 7232 samples, validate on 500 samples
Epoch 1/10
7232/7232 [==============================] - 23s 3ms/step - loss: 0.5712 - acc: 0.7750 - val_loss: 0.4302 - val_acc: 0.8160
Epoch 2/10
7232/7232 [==============================] - 20s 3ms/step - loss: 0.4025 - acc: 0.8202 - val_loss: 0.3647 - val_acc: 0.8340
Epoch 3/10
7232/7232 [==============================] - 20s 3ms/step - loss: 0.3312 - acc: 0.8544 - val_loss: 0.2814 - val_acc: 0.8800
Epoch 4/10
7232/7232 [==============================] - 20s 3ms/step - loss: 0.2438 - acc: 0.9017 - val_loss: 0.2177 - val_acc: 0.9180
Epoch 5/10
7232/7232 [==============================] - 21s 3ms/step - loss: 0.1893 - acc: 0.9262 - val_loss: 0.1955 - val_acc: 0.9300
Epoch 6/10
7232/7232 [==============================] - 21s 3ms/step - loss: 0.1616 - acc: 0.9396 - val_loss: 0.1792 - val_acc: 0.9380
Epoch 7/10
7232/7232 [==============================] - 21s 3ms/step - loss: 0.1421 - acc: 0.9433 - val_loss: 0.1815 - val_acc: 0.9300
Epoch 8/